<a href="https://colab.research.google.com/github/Osmanmertcetin/Fish-Detection-from-Video/blob/main/Train_Fish_Dataset_ipynb_adl%C4%B1_not_defterinin_kopyas%C4%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive            #Google drive hesabının bağlanması.
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

/
Mounted at /content/gdrive


In [ ]:
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive/balik_video/                #Google Drive'da bulunan, veri setini içeren klasör yoluna gelinmesi.

balik_video.flv  dataset_fish.rar


In [ ]:
!cp /mydrive/balik_video/dataset_fish.rar ../           #Klasörde bulunan veri setinin alınması.
!pip install unrar


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!unrar x ../dataset_fish.rar                           #Veri setini içeren rar dosyasının açılması.


UNRAR 5.61 beta 1 freeware      Copyright (c) 1993-2018 Alexander Roshal


Extracting from ../dataset_fish.rar

Creating    annotations                                               OK
Extracting  annotations/A73EGS-P_1.xml                                     0%  OK 
Extracting  annotations/A73EGS-P_3.xml                                     0%  OK 
Extracting  annotations/A73EGS-P_4.xml                                     0%  OK 
Extracting  annotations/A73EGS-P_5.xml                                     0%  OK 
Extracting  annotations/A73EGS-P_7.xml                                     0%  OK 
Extracting  annotations/A73EGSP_6.xml                                      0%  OK 
Extracting  annotations/acanthaluteres_brownii_1.xml                       0%  OK 
Extracting  annotations/acanthaluteres_brownii_2.xml                       0%  OK 
Extracting  annotations/acanthaluteres_brownii_3.xml                      

In [ ]:
import os
import random
import numpy as np
import shutil

print(len(os.listdir('annotations')))     #Veri setinde bulunan fotoğraflara ait xml dosyalarının sayısı.
print(len(os.listdir('images')))          #Veri setinde bulunan fotoğrafların sayısı.

!mkdir test_images                        #Test için kullanılacak fotoğraflar için test_images adında bir klasörün oluşturulması.
!mkdir test_annotations                   #Test için kullanılacak xml dosyaları için test_annotations adında bir klasörün oluşturulması.


random.seed(1234)
idx = random.sample(range(276), 50)

for img in np.array(sorted(os.listdir('images')))[idx]:               #Fotoğraflar rastgele olarak belirlenen oranda test ve eğitim veri kümesi olarak ayrıldı.
    shutil.move('images/'+img, 'test_images/'+img)

for annot in np.array(sorted(os.listdir('annotations')))[idx]:        #Fotoğraflara ait XML dosyaları rastgele olarak belirlenen oranda test ve eğitim veri kümesi olarak ayrıldı.
    shutil.move('annotations/'+annot, 'test_annotations/'+annot)

print(len(os.listdir('annotations')))                                 #Test ve eğitim olarak ayrıldıktan sonra eğitim için kullanılacak xml dosyalarının sayısı.
print(len(os.listdir('images')))                                      #Test ve eğitim olarak ayrıldıktan sonra eğitim için kullanılacak fotoğrafların sayısı.
print(len(os.listdir('test_annotations')))                            #Test ve eğitim olarak ayrıldıktan sonra test için kullanılacak xml dosyalarının sayısı.
print(len(os.listdir('test_images')))                                 #Test ve eğitim olarak ayrıldıktan sonra test için kullanılacak fotoğrafların sayısı.

276
276
226
226
50
50


In [ ]:
import os                                                                     #Gerekli kütüphanelerin import edilmesi.
import numpy as np
import matplotlib.patches as patches
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from PIL import Image
import torchvision
from torchvision import transforms, datasets, models
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import time

In [ ]:
import torch
def generate_box(obj):                              #Fonksiyona gönderilen nesneyi çevreleyen kutu koordinatlarını döndüren fonksiyondur.

    xmin = float(obj.find('xmin').text)
    ymin = float(obj.find('ymin').text)
    xmax = float(obj.find('xmax').text)
    ymax = float(obj.find('ymax').text)

    return [xmin, ymin, xmax, ymax]

adjust_label = 0

def generate_label(obj):

    if obj.find('name').text == "fish":

        return 1 + adjust_label


def generate_target(file):
    with open(file) as f:                                                    #Fonksiyona gönderilen dosya yolunda bulunan xml dosyalarının sırayla açılması.
        data = f.read()                                                      #Dosyanın okunması.
        soup = BeautifulSoup(data, "html.parser")
        objects = soup.find_all("object")                                    #Okunan xml dosyasının içinde tanımlanan tüm objectlerin(nesnelerin) bulunması.

        num_objs = len(objects)                                              #xml dosyası içinde tanımlanan nesnelerin sayısı.

        boxes = []
        labels = []
        for i in objects:                                                    #xml dosyasında bulunan tüm nesneler için sırayla:
            boxes.append(generate_box(i))                                    #generate_box fonksiyonu kullanılarak etrafını çevreleyen kutu koordinatlarının boxes listesine eklenmesi.
            labels.append(generate_label(i))                                 #generate_label fonksiyonu kullanılarak etiketlerin labels listesine eklenmesi.

        boxes = torch.as_tensor(boxes, dtype=torch.float32)                  #Sınırlayıcı kutuları torch tensor türüne dönüştürüldü.
        labels = torch.as_tensor(labels, dtype=torch.int64)                  #Etiketler torch tensor türüne dönüştürüldü.

        target = {}                                                          #target adında boş bir hedef sözlük yapısı oluşturuldu.
        target["boxes"] = boxes                                              #target sözlüğünde boxes anahtar kelimesine karşılık olarak koordinatları içeren boxes listesi verildi.
        target["labels"] = labels                                            #target sözlüğünde labels anahtar kelimesine karşılık olarak etiketleri içeren labels değişkeni verildi.

        return target


In [ ]:
class MaskDataset(object):
    def __init__(self, transforms, path):

        self.transforms = transforms                                            #Kullanılacak dönüşümün tanımlanması.
        self.path = path                                                        #Eğitim veya test klasörünün yolu.
        self.imgs = list(sorted(os.listdir(self.path)))                         #Görüntülerin yolu.


    def __getitem__(self, idx):                                                 #Görüntüyü ve hedefi döndüren fonksiyondur.


        file_image = self.imgs[idx]                                             #Görüntü dosyasının adı.
        file_label = self.imgs[idx][:-3] + 'xml'                                #Görüntü dosyasının adının sonra xml eklenerek etiket dosyasının adı oluşturuldu.
        img_path = os.path.join(self.path, file_image)                          #Görüntü dosyasının tam yolu oluşturuldu (images/balikfotoğrafı.jpg) gibi.

        if 'test' in self.path:                                                 #Gönderilen veri kümesi, test veri kümesi ise etiket dosyasının tam yolu ona göre oluşturuldu.
            label_path = os.path.join("test_annotations/", file_label)
        else:
            label_path = os.path.join("annotations/", file_label)               #Gönderilen veri kümesi, eğitim veri kümesi ise etiket dosyasının tam yolu ona göre oluşturuldu.

        img = Image.open(img_path).convert("RGB")                               #Oluşturulan fotoğraf yolu verilerek fotoğraf açıldı ve RGB türüne dönüştürüldü.

        target = generate_target(label_path)                                    #Hedefin generate_target fonksiyonu kullanılarak oluşturulması.

        if self.transforms is not None:
            img = self.transforms(img)

        return img, target

    def __len__(self):                                                          #Veri kümesinin kaç adet fotoğraf içerdiğini döndüren fonksiyondur.
        return len(self.imgs)

data_transform = transforms.Compose([
        transforms.ToTensor()                                                   #Numpy image türünün torch.Tensor türüne dönüştürülmesi.
    ])

def collate_fn(batch):
    return tuple(zip(*batch))

dataset = MaskDataset(data_transform, 'images/')
test_dataset = MaskDataset(data_transform, 'test_images/')

data_loader = torch.utils.data.DataLoader(dataset, batch_size=4, collate_fn=collate_fn)               #Veri setinin eğitim için kullanılacak kısmının torch DataLoader ile yüklenmesi.
test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=2, collate_fn=collate_fn)     #Veri setinin test için kullanılacak kısmının torch Dataloader ile yüklnmesi.

In [ ]:
def get_model_instance_segmentation(num_classes):                                       #Model olarak faster rcnn resnet50 modelinin seçilmesi.

    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model

In [ ]:
model = get_model_instance_segmentation(2)        #Modele sınıf sayısı olarak 2 verildi(Background ve fish). Kullandığımız faster rcnn resnet50 modelinde veri setimizde
                                                  #bulunan sınıf sayısına ek +1 olarak arkaplan da eklenir. Bunu yapmak nesnelerin doğru bir şekilde sınıflandırılmasını hem de
                                                  #arka planın ayırt edilmesini sağlar.
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')  #Kullanılabilecek GPU mevcutsa GPU eğer değilse CPU'nun kullanılması sağlandı.
model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [ ]:
torch.cuda.is_available()

True

In [ ]:
num_epochs = 10                                                   #Eğitim için kullanılacak epoch sayısı.
params = [p for p in model.parameters() if p.requires_grad]       #Eğitim için gerekli parametrelerin verilmesi.
optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)

In [ ]:
print('----------------------train start--------------------------')                       #Modelin eğitilmesi.
for epoch in range(num_epochs):
    start = time.time()
    model.train()
    i = 0
    epoch_loss = 0
    for imgs, annotations in data_loader:
        i += 1
        imgs = list(img.to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
        loss_dict = model(imgs, annotations)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        epoch_loss += losses
    print(f'epoch : {epoch+1}, Loss : {epoch_loss}, time : {time.time() - start}')

----------------------train start--------------------------
epoch : 1, Loss : 15.045328140258789, time : 73.07235741615295
epoch : 2, Loss : 8.326539993286133, time : 64.99915814399719
epoch : 3, Loss : 6.829817771911621, time : 65.13205480575562
epoch : 4, Loss : 5.903844833374023, time : 65.07701015472412
epoch : 5, Loss : 4.971325874328613, time : 65.26471734046936
epoch : 6, Loss : 4.711432456970215, time : 65.23045587539673
epoch : 7, Loss : 4.22019624710083, time : 65.18818736076355
epoch : 8, Loss : 4.139559268951416, time : 65.23938226699829
epoch : 9, Loss : 4.100071907043457, time : 65.15059185028076
epoch : 10, Loss : 4.193971633911133, time : 65.15284180641174


In [ ]:
torch.save(model.state_dict(),f'model_balik_tespiti.pt')        #Eğitilen modelin kaydedilmesi.

In [ ]:
                                                                #Eğitilen modelin Google Drive'a kaydedilmesi.
!cp -r './model_10.pt' /mydrive/balik_video/